# Mixture Of Reasoning Experts (MoRE)

## Bước 1: Như đã thực hiện trong hình





---

**Mixture of Reasoning Experts (MORE)**  
Phần này giới thiệu khung làm việc **Mixture of Reasoning Experts (MORE)**, bao gồm cách xây dựng các chuyên gia suy luận đa dạng, cách kết hợp chúng, và cách dự đoán tính chính xác của câu trả lời.  

**3.1 Các chuyên gia suy luận chuyên biệt**  
Bước đầu tiên của hệ thống **MORE** là thu thập một tập hợp đa dạng các mô hình chuyên biệt để có thể kết hợp điểm mạnh của chúng thông qua chiến lược tổ hợp. Mặc dù có nhiều cách để xây dựng các mô hình QA chuyên biệt, chúng tôi thiết kế các chuyên gia suy luận chuyên biệt thông qua việc sử dụng lời nhắc (prompting) cho một mô hình ngôn ngữ lớn (LLM), vì chúng đạt độ chính xác hàng đầu trong nhiều tác vụ suy luận.  

Chúng tôi chuyên biệt hóa mô hình **Codex** (Chen et al., 2021) cho các loại suy luận khác nhau bằng bốn phương pháp nhắc chuyên biệt (các lời nhắc mẫu được liệt kê trong Phụ lục, Hình 3):  

- **Chuyên gia thực tế (Factual expert)** với lời nhắc được tăng cường bởi truy xuất thông tin. Theo Si và cộng sự (2023a), với mỗi câu hỏi, chúng tôi truy xuất 10 đoạn văn bản liên quan nhất từ Wikipedia bằng công cụ **Contriever** (Izacard et al., 2022) và thêm chúng vào lời nhắc ngay trước câu hỏi.  
- **Chuyên gia suy luận đa bước (Multihop expert)** với lời nhắc theo phương pháp **Chain-of-Thought (CoT)** (Wei et al., 2022b). Chúng tôi thêm các lý giải được viết tay sau mỗi câu hỏi mẫu trong lời nhắc để gợi ra quy trình suy luận nhiều bước cho câu hỏi.  
- **Chuyên gia toán học (Math expert)** cũng sử dụng lời nhắc CoT. Chúng tôi thêm các giải thích đi kèm được cung cấp trong **GSM8K** sau mỗi câu hỏi mẫu trong lời nhắc để gợi ra các bước suy luận tương tự cho câu hỏi.  
- **Chuyên gia thường thức (Commonsense expert)** với lời nhắc kiến thức được tạo tự động (generated knowledge prompting) (Liu et al., 2021). Chúng tôi tạo ra 10 mẩu thông tin liên quan đến mỗi câu hỏi bằng mô hình **Codex** và thêm chúng vào lời nhắc ngay trước câu hỏi.  

Sau khi thu thập dự đoán từ mỗi chuyên gia, chúng tôi huấn luyện một bộ phân loại để chọn ra câu trả lời tốt nhất. Điều này cho phép **MORE** kết hợp bốn mô hình chuyên biệt mà không cần biết trước kiểu suy luận của câu hỏi.  

---

## Bước 2: Chọn câu hỏi bằng random forest classifier



**3.2 Kết hợp thông qua lựa chọn câu trả lời**  
Phương pháp **MORE** kết hợp điểm mạnh của các chuyên gia suy luận bằng cách sử dụng một bộ phân loại ngẫu nhiên rừng (random forest classifier) dựa trên đặc trưng để đánh giá mỗi câu trả lời. Điểm số này được dùng để chọn câu trả lời cuối cùng và xác định khi nào nên bỏ qua câu hỏi.

 **Bộ đặc trưng**  (feature set for model random forcast)
- **Các đặc trưng thiết kế thủ công**:
  - **Loại chuyên gia** (expert type).  
  - **Đặc điểm câu hỏi**: từ khóa câu hỏi, độ dài, có chứa giá trị số hay không.  
  - **Đặc điểm câu trả lời**: độ tự tin, độ dài, mức độ trùng khớp từ khóa với câu hỏi, ngữ cảnh, và lý giải.  
  - **Đồng thuận giữa các chuyên gia**: tần suất câu trả lời được dự đoán giống nhau bởi các chuyên gia, mức độ trùng khớp từ khóa giữa các câu trả lời.  

 **Huấn luyện bộ phân loại**  
- **Dữ liệu huấn luyện**: Giữ lại 100 mẫu từ mỗi bộ dữ liệu QA (tổng cộng 1200 mẫu) làm dữ liệu huấn luyện.  
- **Mục tiêu**: Huấn luyện bộ phân loại để dự đoán xem câu trả lời của mô hình chuyên gia có chính xác hay không (nhị phân).  
- Trong quá trình suy luận (inference), câu trả lời có điểm số cao nhất sẽ được chọn. Nếu điểm số thấp hơn ngưỡng, hệ thống sẽ bỏ qua không trả lời.  

 **Các thí nghiệm khác**  
- Thử nghiệm với các bộ phân loại khác và tinh chỉnh các mô hình ngôn ngữ tiền huấn luyện như BERT, nhưng không hiệu quả bằng random forest.  



## Library

In [ ]:
!pip install instructor datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.7.1
    Uninstalling jiter-0.7.1:
      Successfully uninstalled jiter-0.7.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

## Pre data GSM8K

In [ ]:
import json
import random
import instructor
import re
from pathlib import Path
from typing import List, TextIO

import openai
from datasets import load_dataset
# from fire import Fire
from pydantic import BaseModel
from tqdm import tqdm


def remove_substrings_with_double_angle_brackets(input_string):
    # Define the pattern to match substrings within double angled brackets
    pattern = r"<<[^>]+>>"
    # Use the sub() function from the re module to replace matching substrings with an empty string
    result = re.sub(pattern, "", input_string)
    return result


class ReasonSample(BaseModel):
    question: str
    explanation: str = ""
    answer: str = ""
    wrong_explanation: str = ""
    wrong_answer: str = ""
    pred: str = ""


class ReasonData(BaseModel):
    samples: List[ReasonSample]

    @classmethod
    def load(cls, path: str):
        samples = []
        with open(path) as f:
            for line in f:
                raw = json.loads(line)
                samples.append(ReasonSample(**raw))

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_test(cls, path: str = "gsm8k", subset: str = "main", split="test"):
        samples = []
        for raw in load_dataset(path, subset, split=split):
            explanation, answer = raw["answer"].split("####")
            explanation = remove_substrings_with_double_angle_brackets(explanation)
            samples.append(
                ReasonSample(
                    question=raw["question"].strip(),
                    explanation=explanation.strip(),
                    answer=answer.strip(),
                )
            )

        return cls(samples=samples)

    @classmethod
    def load_gsm8k_incoherent_objects(cls, split: str = "test", sample: bool = False):
        if split == "train" and sample:
            samples = [
                ReasonSample(
                    question="There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
                    explanation="There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.",
                    answer="6",
                    wrong_explanation="There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.",
                    wrong_answer="21",
                ),
                ReasonSample(
                    question="If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot??",
                    explanation="There are originally 3 cars. Then 2 more cars arrive. Now 3 + 2 = 5 cars are in the parking lot.",
                    answer="5",
                    wrong_explanation="There are originally 3 + 2 = 5 cars. Then 3 more cars arrive. Now 2 cars are in the parking lot",
                    wrong_answer="2",
                ),
                ReasonSample(
                    question="Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
                    explanation="Originally, Leah had 32 chocolates and her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39 pieces left in total.",
                    answer="39",
                    wrong_explanation="Originally, Leah had 32 + 42 = 74 chocolates and her sister had 32. So in total they had 74 - 35 = 39. After eating 35, they had 42 pieces left in total",
                    wrong_answer="42",
                ),
                ReasonSample(
                    question="Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
                    explanation="Jason had 20 lollipops originally. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8 lollipops.",
                    answer="8",
                    wrong_explanation="Jason had 20 - 12 = 8 lollipops originally. Then he had 20 after giving some to Denny. So he gave Denny 12 lollipops",
                    wrong_answer="12",
                ),
            ]
            return cls(samples=samples)
        else:
            return cls.load_gsm8k_test()

    @classmethod
    def load_from_name(cls, name: str, **kwargs):
        if name == "gsm8k":
            return cls.load_gsm8k_incoherent_objects(**kwargs)
        else:
            raise KeyError(name)


class Prompter(BaseModel):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Answer: "
        return prompt

    @staticmethod
    def get_answer(text: str) -> str:
        parts = text.split("Answer: ")
        if len(parts) >= 2:
            return parts[1]
        else:
            return text


class ChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt

    def get_explanation(self, text: str) -> str:
        assert self is not None
        return text.split("\nAnswer: ")[0]


class ContrastiveChainThoughtPrompter(Prompter):
    def run(self, data_train: ReasonData, sample_test: ReasonSample) -> str:
        prompt = ""
        for sample in data_train.samples:
            prompt += f"Question: {sample.question}\n"
            prompt += f"Explanation: {sample.explanation}\n"
            prompt += f"Answer: {sample.answer}\n"
            prompt += f"Wrong explanation: {sample.wrong_explanation}\n"
            prompt += f"Wrong Answer: {sample.wrong_answer}\n\n"

        prompt += f"Question: {sample_test.question}\n"
        prompt += "Explanation: "
        return prompt


def select_prompter(name: str):
    if name == "standard":
        return Prompter()
    if name == "cot":
        return ChainThoughtPrompter()
    if name == "contrast_cot":
        return ContrastiveChainThoughtPrompter()
    else:
        raise KeyError(name)


In [ ]:
data_test = ReasonData.load_from_name('gsm8k', split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
data_test.samples[0].question

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [ ]:
data_train = ReasonData.load_from_name('gsm8k', split="train", sample=True)

In [ ]:
data_train.samples[0]

ReasonSample(question='There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?', explanation='There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.', answer='6', wrong_explanation='There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.', wrong_answer='21', pred='')

## Coding Step 1 : Mixture of Reasoning Experts

In [ ]:
import instructor
from pydantic import BaseModel
from collections import defaultdict, Counter
import openai
import asyncio
from textwrap import dedent
import math

client = openai.AsyncOpenAI(api_key='sk-U7ti37t3osaeWme7m88923xFY2UMGVQLkeydTAywi3xI7TOP',
                                              base_url="https://api.chatanywhere.tech/v1")
client = instructor.from_openai(client)

class Response(BaseModel):
    chain_of_thought: list[str]
    answer: int


class ResponseScore(BaseModel):
    query: str
    response: Response
    score: float

    def format_response(self):
        return dedent(
            f"""
            Q: {self.query}
            A: {''.join(self.response.chain_of_thought)}. Therefore the answer is {self.response.answer}.
            """
        )
def cosine_similarity(vec1: list[float], vec2: list[float]):
    dot_product = sum(a * b for a, b in zip(vec1, vec2))
    magnitude1 = math.sqrt(sum(a * a for a in vec1))
    magnitude2 = math.sqrt(sum(b * b for b in vec2))

    if magnitude1 * magnitude2 == 0:
        return 0  # Handle the case of zero vectors

    return dot_product / (magnitude1 * magnitude2)


def score_repetitiveness(prediction: Response):

    embedding = openai.OpenAI(api_key='sk-U7ti37t3osaeWme7m88923xFY2UMGVQLkeydTAywi3xI7TOP',
                                              base_url="https://api.chatanywhere.tech/v1").embeddings.create(input=prediction.chain_of_thought, model="text-embedding-3-small"
    )
    embedding = [item.embedding for item in embedding.data]
    print(len(embedding))
    ttl = 0
    num_comparisons = 0
    for idx in range(len(embedding)):
        for idx2 in range(idx + 1, len(embedding)):
            ttl += cosine_similarity(embedding[idx], embedding[idx2])
            num_comparisons += 1
    print(num_comparisons)
    return 2*ttl/(num_comparisons)*(num_comparisons - 1) if num_comparisons > 0 else 0

async def generate_cot_response(query: str) -> tuple[Response, str]:
    return (
        await client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": query}],
            response_model=Response,
            temperature=0.4,
        ),
        query,
    )


async def generate_batch_cot_responses(
    queries: list[str], m: int
) -> list[tuple[Response, str]]:
    coros = [generate_cot_response(query) for query in queries for _ in range(m)]
    return await asyncio.gather(*coros)

def score_entropy(predictions: list[Response]):
    counter = Counter([prediction.answer for prediction in predictions])

    prob = [counter[i] / len(predictions) for i in counter]

    numer = -sum([p * math.log(p) for p in prob])
    denom = math.log(len(predictions))

    return numer / denom


def score_responses(
    predictions: list[tuple[Response, str]], trade_off_param: float
) -> list[ResponseScore]:
    query_to_responses: dict[str, list[Response]] = defaultdict(list)
    for prediction, query in predictions:
        query_to_responses[query].append(prediction)

    query_to_entropy = {
        query: score_entropy(predictions)
        for query, predictions in query_to_responses.items()
    }

    return [
        ResponseScore(
            query=query,
            response=prediction,
            score=query_to_entropy[query]
            + trade_off_param * score_repetitiveness(prediction),
        )
        for prediction, query in predictions
    ]


def get_top_k_examples(queries: list[ResponseScore], k: int):
    """
    This gets the top k responses that have the minimum possible score
    """
    sorted_responses = sorted(queries, key=lambda x: x.score)
    return sorted_responses[:k]


async def generate_answer_with_examples(query: str, examples: list[ResponseScore]):
    formatted_examples = "\n".join([example.format_response() for example in examples])
    return await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": dedent(
                    f"""
                You are a world class AI system that excels at answering user queries

                <query>
                {query}
                </query>

                <examples>
                {formatted_examples}
                </examples>
                """
                ),
            }
        ],
        response_model=Response,
    )


async def generate_final_answers(
    query: str, examples: list[ResponseScore], number_samples: int
):
    coros = [
        generate_answer_with_examples(query, examples) for _ in range(number_samples)
    ]

    return await asyncio.gather(*coros)

In [ ]:
import nest_asyncio
loop = asyncio.get_event_loop()
# Apply nest_asyncio to the loop
nest_asyncio.apply(loop)

# Run the coroutine using the loop's run_until_complete method
m = 2  # Number of Reasoning Chains per example ( Step 1 )
k = 3  # Number of Examples to include in final prompt (Step 2)
n = 2  # Number of Reasoning Chains For Self-Consistency ( Step 2 )

# Step 1 : Generate the examples
query = (
    "The schools debate team had 5 boys and 40 girls on it. "
    "If they were split into groups of 9 how many groups "
    "could they make?"
)

example_questions = [
    (
        "Debby's class is going on a field trip to the zoo. "
        "If each van can hold 4 people and there are 2 students "
        "and 6 adults going, how many vans will they need?"
    ),
    (
        "Nancy had 80 files on her computer. She deleted 31 of "
        "them and put the rest into folders with 7 files in each "
        "one. How many folders did Nancy end up with?"
    ),
    (
        "At the arcade, Tom won 32 tickets playing 'whack a mole' "
        "and 25 tickets playing 'skee ball'. If he spent 7 of his "
        "tickets on a hat, how many tickets does Tom have left?"
    ),
]

m = 2  # Number of Reasoning Chains per example ( Step 1 )
k = 3  # Number of Examples to include in final prompt (Step 2)
n = 2  # Number of Reasoning Chains For Self-Consistency ( Step 2 )

# Step 1 : Generate the examples
responses = loop.run_until_complete(generate_batch_cot_responses(example_questions, m))
scored_responses = score_responses(responses, 0.2)

chosen_examples = get_top_k_examples(scored_responses, k)

# Step 2 : Run Self-Consistency
final_responses = loop.run_until_complete(generate_final_answers(query, chosen_examples, 1))

3
3
4
6
5
10
4
6
5
10
5
10


In [ ]:
responses

[(Response(chain_of_thought=['Calculate the total number of people going on the trip: 2 students + 6 adults = 8 people.', 'Determine how many vans are needed by dividing the total number of people by the capacity of each van: 8 people ÷ 4 people per van = 2 vans.', 'Since the number of vans must be a whole number, and 8 is exactly divisible by 4, no additional van is needed.'], answer=2),
  "Debby's class is going on a field trip to the zoo. If each van can hold 4 people and there are 2 students and 6 adults going, how many vans will they need?"),
 (Response(chain_of_thought=['Total people going on the trip = number of students + number of adults = 2 + 6 = 8', 'Each van can hold 4 people.', 'To find the number of vans needed, divide the total number of people by the capacity of each van: 8 / 4 = 2.', 'Since 8 is exactly divisible by 4, they will need 2 vans.'], answer=2),
  "Debby's class is going on a field trip to the zoo. If each van can hold 4 people and there are 2 students and 6 

In [ ]:
chosen_examples

[ResponseScore(query="Debby's class is going on a field trip to the zoo. If each van can hold 4 people and there are 2 students and 6 adults going, how many vans will they need?", response=Response(chain_of_thought='First, we need to determine the total number of people going on the field trip. There are 2 students and 6 adults, so we add these together: 2 + 6 = 8 people in total. Next, we know that each van can hold 4 people. To find out how many vans are needed, we divide the total number of people by the capacity of each van: 8 ÷ 4 = 2 vans. Therefore, they will need 2 vans for the trip.', answer=2), score=0.0),
 ResponseScore(query="Debby's class is going on a field trip to the zoo. If each van can hold 4 people and there are 2 students and 6 adults going, how many vans will they need?", response=Response(chain_of_thought='First, we need to calculate the total number of people going on the field trip. There are 2 students and 6 adults, so we add these numbers together: 2 + 6 = 8 pe

In [ ]:
from openai import OpenAI
from pydantic import BaseModel, Field
import instructor
from textwrap import dedent

client1 = OpenAI(api_key='sk-RYiDonFCU85Bfr242X2POvCBhlhfHlz8knZQoVjWVJw180s6',
                                              base_url="https://api.chatanywhere.tech/v1")
client1 = instructor.from_openai(client1)


class MultihopExpert(BaseModel):
    chain_of_thought: list[str]
    answer: str


class FactualExpert(BaseModel):
    chain_of_thought: list[str]
    answer: str

class MathExpert(BaseModel):
    chain_of_thought: list[str] = Field(description="Incorrect reasoning for the answer")
    answer: str

class CommonsenseExpert(BaseModel):
    chain_of_thought: list[str]
    answer: str

class ModelScore(BaseModel):
    score: float = Field(ge=0, lt=1)


def query_factual_expert(query: str, evidence: list[str]):
    formatted_evidence = "\n-".join(evidence)
    return client1.chat.completions.create(
        model="gpt-4o-mini",
        response_model=FactualExpert,
        messages=[
            {
                "role": "system",
                "content": dedent(
                    f"""
                <query>
                {query}
                </query>

                <evidences>
                {formatted_evidence}
                </evidences>
                """
                ),
            }
        ],
    )

def query_multihop_expert(query: str):
    return client1.chat.completions.create(
        model="gpt-4o-mini",
        response_model=MultihopExpert,
        messages=[
            {
                "role": "system",
                "content": dedent(
                    f"""
                <query>
                {query}
                </query>
                """
                ),
            }
        ],
    )

def query_math_expert(query: str, sample):
  return client1.chat.completions.create(
      model='gpt-4o-mini',
      response_model=MathExpert,
      messages=[
          {
                "role": "system",
                "content": dedent(
                    f"""
            <prompt>
                <role>system</role>
                <context>
                You are an expert question answering AI System.

                You are about to be given some examples of incorrect
                and correct reasoning for a question. You will then
                be asked to correctly reason through another question
                to generate a valid response.
                </context>

                <question>{sample.question}</question>

                <Explanations>
                    {sample.explanation}
                    {sample.wrong_explanation}
                </Explanations>
                <question>{query}</question>
            </prompt>
            """
                ),
            }
      ]

  )

def score_answer(query: str, answer: str):
    return client1.chat.completions.create(
        model="gpt-4o-mini",
        response_model=ModelScore,
        messages=[
            {
                "role": "system",
                "content": """You are a helpful assistant that scores
                answers based on well they are able to answer a
                specific user query""",
            },
            {
                "role": "user",
                "content": f"""
                <user query>
                {query}
                </user query>

                <response>
                {answer}
                </response>
                """,
            },
        ],
    )

def query_commonsense_expert(query: str, evidence: list[str]):
    formatted_evidence = "\n-".join(evidence)



In [ ]:
data_train.samples[0]

ReasonSample(question='There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?', explanation='There are 15 trees originally. Then there were 21 trees after the Grove workers planted some more. So there must have been 21 - 15 = 6 trees that were planted.', answer='6', wrong_explanation='There are 21 - 15 = 6 trees originally. Then there were 15 trees after the Grove workers planted some more. So there must have been 21 trees that were planted.', wrong_answer='21', pred='')

In [ ]:
if __name__ == "__main__":
    # query = """Who's the original singer of Help Me Make It
    # Through The Night?"""

    evidences = [
        """Help Me Make It Through The Night is a country
        music ballad written and composed by Kris Kristofferson
        and released on his 1970 album 'Kristofferson'"""
    ]

    threshold = 0.8

    factual_expert_output = query_factual_expert(query, evidences)
    print(factual_expert_output.model_dump_json(indent=2))

    multihop_expert_output = query_multihop_expert(query)
    print(multihop_expert_output.model_dump_json(indent=2))

    math_expert_output = query_math_expert(query, data_train.samples[0])
    print(math_expert_output.model_dump_json(indent=2))

    factual_expert_score = score_answer(query, factual_expert_output.answer)
    multihop_expert_score = score_answer(query, multihop_expert_output.answer)
    math_expert_score = score_answer(query, math_expert_output.answer)

    if max(factual_expert_score.score, multihop_expert_score.score) < threshold:
        answer = "Abstaining from responding"
    elif factual_expert_score.score > multihop_expert_score.score:
        answer = factual_expert_output.answer
    else:
        answer = multihop_expert_output.answer

    print(answer)

{
  "chain_of_thought": [
    "Calculate the total number of students in the debate team by adding the number of boys and girls.",
    "Total students = 5 boys + 40 girls = 45 students.",
    "Next, determine how many groups of 9 can be formed with 45 students by dividing 45 by 9.",
    "Groups = 45 students / 9 students per group."
  ],
  "answer": "5 groups"
}
{
  "chain_of_thought": [
    "First, we need to determine the total number of students on the debate team.",
    "There are 5 boys and 40 girls. So, total students = 5 + 40 = 45.",
    "Next, we want to split these 45 students into groups of 9.",
    "To find the number of groups, we divide the total number of students by the group size: 45 / 9.",
    "Calculating this gives us 5."
  ],
  "answer": "5"
}
{
  "chain_of_thought": [
    "The debate team has a total of 5 boys and 40 girls, which sums up to 5 + 40 = 45 members.",
    "If the team is split into groups of 9, we need to determine how many full groups of 9 can be forme

In [ ]:
print(factual_expert_output.model_dump_json(indent=2),factual_expert_score)

{
  "answer": "The debate team has a total of 5 boys + 40 girls = 45 members. If they split into groups of 9, we can divide 45 by 9. \n\n45 ÷ 9 = 5. \n\nThus, they can make 5 groups."
} score=0.9


In [ ]:
print(multihop_expert_output.model_dump_json(indent=2),multihop_expert_score)

{
  "chain_of_thought": "The song 'Help Me Make It Through The Night' was originally performed by the artist Sammi Smith. It was released in 1970 and became a significant hit.",
  "answer": "Sammi Smith"
} score=0.8


In [ ]:
print(math_expert_output.model_dump_json(indent=2),math_expert_score)

{
  "chain_of_thought": "The song 'Help Me Make It Through The Night' was originally sung by Sammi Smith in 1970, making her the original artist. It's important to identify the time frame as well as the artist's claim to the song.",
  "correct_answer": "Sammi Smith"
} score=0.9


In [ ]:
query = "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [ ]:
factual_expert_output = query_factual_expert(query, evidences)
print(factual_expert_output.model_dump_json(indent=2))

multihop_expert_output = query_multihop_expert(query)
print(multihop_expert_output.model_dump_json(indent=2))

math_expert_output = query_math_expert(query, data_train.samples[0])
print(math_expert_output.model_dump_json(indent=2))

factual_expert_score = score_answer(query, factual_expert_output.answer)
multihop_expert_score = score_answer(query, multihop_expert_output.answer)
math_expert_score = score_answer(query, math_expert_output.answer)

{
  "chain_of_thought": [
    "Janet’s ducks lay 16 eggs per day.",
    "She eats 3 eggs for breakfast every morning, leaving her with 16 - 3 = 13 eggs.",
    "She bakes 4 muffins with 4 eggs, leaving her with 13 - 4 = 9 eggs.",
    "The remainder of the eggs that she sells is 9 eggs.",
    "She sells each egg for $2, so her total earnings from selling the eggs is 9 * 2 = 18."
  ],
  "answer": "18"
}
{
  "chain_of_thought": [
    "Janet has 16 duck eggs.",
    "She eats 3 eggs for breakfast, so remaining eggs after breakfast = 16 - 3 = 13 eggs.",
    "She bakes muffins with 4 eggs, so remaining eggs after baking = 13 - 4 = 9 eggs.",
    "Janet sells the remaining eggs (9) at the farmers' market for $2 each.",
    "Total earnings from selling eggs = 9 eggs * $2/egg = $18."
  ],
  "answer": "18"
}
{
  "chain_of_thought": [
    "Janet's ducks lay 16 eggs every day.",
    "She eats 3 eggs for breakfast, which means she has 16 - 3 = 13 eggs left.",
    "She bakes muffins with 4 eggs, so now

In [ ]:
m = 2  # Number of Reasoning Chains per example ( Step 1 )
k = 3  # Number of Examples to include in final prompt (Step 2)
n = 2  # Number of Reasoning Chains For Self-Consistency ( Step 2 )

# Step 1 : Generate the examples
responses = loop.run_until_complete(generate_batch_cot_responses(example_questions, m))
scored_responses = score_responses(responses, 0.2)

chosen_examples = get_top_k_examples(scored_responses, k)

# Step 2 : Run Self-Consistency
final_responses = loop.run_until_complete(generate_final_answers(query, chosen_examples, 1))

3
3
3
3
5
10
4
6
5
10
5
10


In [ ]:
commonsense_expert_output = final_responses[0]
math_expert_score = score_answer(query, commonsense_expert_output.answer)

In [ ]:
print(factual_expert_output.model_dump_json(indent=2),factual_expert_score)
print(multihop_expert_output.model_dump_json(indent=2),multihop_expert_score)
print(math_expert_output.model_dump_json(indent=2),math_expert_score)
print(commonsense_expert_output.model_dump_json(indent=2),math_expert_score)

{
  "chain_of_thought": [
    "Janet’s ducks lay 16 eggs per day.",
    "She eats 3 eggs for breakfast every morning, leaving her with 16 - 3 = 13 eggs.",
    "She bakes 4 muffins with 4 eggs, leaving her with 13 - 4 = 9 eggs.",
    "The remainder of the eggs that she sells is 9 eggs.",
    "She sells each egg for $2, so her total earnings from selling the eggs is 9 * 2 = 18."
  ],
  "answer": "18"
} score=0.1
{
  "chain_of_thought": [
    "Janet has 16 duck eggs.",
    "She eats 3 eggs for breakfast, so remaining eggs after breakfast = 16 - 3 = 13 eggs.",
    "She bakes muffins with 4 eggs, so remaining eggs after baking = 13 - 4 = 9 eggs.",
    "Janet sells the remaining eggs (9) at the farmers' market for $2 each.",
    "Total earnings from selling eggs = 9 eggs * $2/egg = $18."
  ],
  "answer": "18"
} score=0.0
{
  "chain_of_thought": [
    "Janet's ducks lay 16 eggs every day.",
    "She eats 3 eggs for breakfast, which means she has 16 - 3 = 13 eggs left.",
    "She bakes muffins

In [ ]:
rep = []
rep.append((factual_expert_output))
rep.append((multihop_expert_output))
rep.append((math_expert_output))
rep.append((commonsense_expert_output))

## Coding Step 2:  Chọn câu hỏi đúng bằng random forest classifier

Việc sử dụng random forest tương đối khó khi cần dữ liệu để huấn luyện một model sau đó mới có thể phân loại câu hỏi chính xác nhất

xem thêm [code](https://github.com/NoviScl/MoRE/tree/main):

Thay vào đó có thể sử dụng chọn câu hỏi bằng sử dụng entropy and Repetitiveness

In [ ]:
rep

[FactualExpert(chain_of_thought='To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.', answer='5'),
 MultihopExpert(chain_of_thought='First, we need to determine the total number of students in the debate team. There are 5 boys and 40 girls, so we add them together: 5 + 40 = 45 students in total. Next, we want to find out how many groups of 9 can be made from these 45 students. To do this, we divide the total number of students by the group size: 45 ÷ 9 = 5 groups. Therefore, the number of groups that can be made is 5.', answer='5'),
 MathExpert(chain_of_thought='The total number of students on the debate team is 5 boys + 40 girls = 45 students. If th

In [ ]:
responses[0]

(Response(chain_of_thought='First, we need to determine the total number of people going on the field trip. There are 2 students and 6 adults, so we add these together: 2 + 6 = 8 people in total. Next, we know that each van can hold 4 people. To find out how many vans are needed, we divide the total number of people by the capacity of each van: 8 ÷ 4 = 2 vans. Therefore, they will need 2 vans for the trip.', answer=2),
 "Debby's class is going on a field trip to the zoo. If each van can hold 4 people and there are 2 students and 6 adults going, how many vans will they need?")

In [ ]:
class ResponseScore(BaseModel):
    query: str
    response: Response
    score: float

    def format_response(self):
        return dedent(
            f"""
            Q: {self.query}
            A: {''.join(self.response.chain_of_thought)}. Therefore the answer is {self.response.answer}.
            """
        )

In [ ]:
for re in rep:
  print(re.chain_of_thought)

To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.
First, we need to determine the total number of students in the debate team. There are 5 boys and 40 girls, so we add them together: 5 + 40 = 45 students in total. Next, we want to find out how many groups of 9 can be made from these 45 students. To do this, we divide the total number of students by the group size: 45 ÷ 9 = 5 groups. Therefore, the number of groups that can be made is 5.
The total number of students on the debate team is 5 boys + 40 girls = 45 students. If they are split into groups of 9, we need to divide 45 by 9. The calculation is 45 ÷ 9 = 5. Therefore, they can make 5 groups of 9

In [ ]:
def score_entropy(predictions: list[Response]):
    counter = Counter([prediction for prediction in predictions])

    prob = [counter[i] / len(predictions) for i in counter]

    numer = -sum([p * math.log(p) for p in prob])
    denom = math.log(len(predictions))

    return numer / denom

In [ ]:
query_to_responses = {
    re.answer: re.chain_of_thought  # Store the entire Response object
    for re in rep
}

In [ ]:
for query, responses in query_to_responses.items():
  print(query)
  print(responses)

To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.
chain_of_thought='To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.' answer='5'
First, we need to determine the total number of students in the debate team. There are 5 boys and 40 girls, so we add them together: 5 + 40 = 45 students in total. Next, we want to fin

In [ ]:
def score_repetitiveness(repo):

    embedding = openai.OpenAI(api_key='sk-U7ti37t3osaeWme7m88923xFY2UMGVQLkeydTAywi3xI7TOP',
                                              base_url="https://api.chatanywhere.tech/v1").embeddings.create(input=repo.chain_of_thought, model="text-embedding-3-small"
    )
    embedding = [item.embedding for item in embedding.data]
    ttl = 0
    num_comparisons = 0
    for idx in range(len(embedding)):
        for idx2 in range(idx + 1, len(embedding)):
            ttl += cosine_similarity(embedding[idx], embedding[idx2])
            num_comparisons += 1
    return 2*ttl/(num_comparisons)*(num_comparisons - 1) if num_comparisons > 0 else 0

In [ ]:
def score_entropy(predictions):
    counter = Counter([prediction.answer for prediction in predictions])

    prob = [counter[i] / len(predictions) for i in counter]

    numer = -sum([p * math.log(p) for p in prob])
    denom = math.log(len(predictions))

    return numer / denom

In [ ]:
score2 = score_entropy(rep)


In [ ]:
score2

0.4056390622295664

In [ ]:

for re in rep:
  print(re.chain_of_thought)
  print(re.answer)
  score1 = score_repetitiveness(re)
  print(score1 + score2)


['Janet’s ducks lay 16 eggs per day.', 'She eats 3 eggs for breakfast every morning, leaving her with 16 - 3 = 13 eggs.', 'She bakes 4 muffins with 4 eggs, leaving her with 13 - 4 = 9 eggs.', 'The remainder of the eggs that she sells is 9 eggs.', 'She sells each egg for $2, so her total earnings from selling the eggs is 9 * 2 = 18.']
18
10.673412190874297
['Janet has 16 duck eggs.', 'She eats 3 eggs for breakfast, so remaining eggs after breakfast = 16 - 3 = 13 eggs.', 'She bakes muffins with 4 eggs, so remaining eggs after baking = 13 - 4 = 9 eggs.', "Janet sells the remaining eggs (9) at the farmers' market for $2 each.", 'Total earnings from selling eggs = 9 eggs * $2/egg = $18.']
18
10.010257086625494
["Janet's ducks lay 16 eggs every day.", 'She eats 3 eggs for breakfast, which means she has 16 - 3 = 13 eggs left.', 'She bakes muffins with 4 eggs, so now she has 13 - 4 = 9 eggs remaining.', "Janet sells those 9 eggs at the farmers' market for $2 each, so she makes 9 * 2 = 18 dolla

In [ ]:
class ResponseScore(BaseModel):
    query: str
    response: Response
    score: float

    def format_response(self):
        return dedent(
            f"""
            Q: {self.query}
            A: {''.join(self.response)}. Therefore the answer is {self.response.answer}.
            """
        )

In [ ]:
a = []
for re in rep:
  print(re.chain_of_thought)
  print(re.answer)
  query = query
  score = query_to_entropy[re.answer] + 0.2 * score_repetitiveness(re.chain_of_thought)
  print(score)
  # a.append(ResponseScore(query='', response='t', score=score))

To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.
5
0.4967648485783058
First, we need to determine the total number of students in the debate team. There are 5 boys and 40 girls, so we add them together: 5 + 40 = 45 students in total. Next, we want to find out how many groups of 9 can be made from these 45 students. To do this, we divide the total number of students by the group size: 45 ÷ 9 = 5 groups. Therefore, the number of groups that can be made is 5.
5
0.4967648485783058
The total number of students on the debate team is 5 boys + 40 girls = 45 students. If they are split into groups of 9, we need to divide 45 by 9. The calculation is 45 ÷ 9 =

In [ ]:
def score_responses(
    predictions, trade_off_param: float
) -> list[ResponseScore]:
    query_to_responses: dict[str, list[Response]] = defaultdict(list)
    for prediction, query in predictions:
        query_to_responses[query].append(prediction)

    # Change here: Use the actual response objects for each query
    # instead of the original 'predictions' list
    print(query_to_responses)
    query_to_entropy = {
        query: responses
        for query, responses in query_to_responses.items()
    }
    print(query_to_entropy)
    query_to_entropy = {
        query: score_entropy(responses)
        for query, responses in query_to_responses.items()
    }

    return [
        ResponseScore(
            query=query,
            response=prediction,
            score=query_to_entropy[query]
            + trade_off_param * score_repetitiveness(prediction),
        )
        for prediction, query in predictions
    ]

In [ ]:
scored_responses = score_responses(rep, 0.2)

chosen_examples = get_top_k_examples(scored_responses, k)

# Step 2 : Run Self-Consistency
final_responses = asyncio.run(generate_final_answers(query, chosen_examples, n))

c = Counter([response.answer for response in final_responses])
answer = c.most_common(1)[0][0]

print(answer)

defaultdict(<class 'list'>, {('answer', '5'): [('chain_of_thought', 'To find the total number of members in the debate team, we add the number of boys and girls together. There are 5 boys and 40 girls, so the total is: 5 + 40 = 45. Next, we need to determine how many groups of 9 can be made from these 45 members. We do this by dividing the total number of members by the size of each group: 45 ÷ 9 = 5. Therefore, the team can make 5 groups of 9 members each.'), ('chain_of_thought', 'First, we need to determine the total number of students in the debate team. There are 5 boys and 40 girls, so we add them together: 5 + 40 = 45 students in total. Next, we want to find out how many groups of 9 can be made from these 45 students. To do this, we divide the total number of students by the group size: 45 ÷ 9 = 5 groups. Therefore, the number of groups that can be made is 5.')], ('answer', '5 groups'): [('chain_of_thought', 'The total number of students on the debate team is 5 boys + 40 girls = 

AttributeError: 'tuple' object has no attribute 'answer'